## Picture Processing and Object Detection

In [10]:
import cv2
import numpy as np
import pandas as pd

In [11]:
#load yolo
net = cv2.dnn.readNet("yolov3.weights","yolov3.cfg")

classes = []
with open("coco.names","r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

### Demo batch processing

In [12]:
imgs=["img/kitchen.jpg",
      "img/bathroom.jpg",
      "img/apple-pizza.jpg",
      "img/people1.png",
      "img/people2.png"]

In [13]:
photos=[]
labels=[]
for p in imgs:
    img = cv2.imread(p)
    img = cv2.resize(img, None, fx=0.6, fy=0.6)
    height,width,channels = img.shape
    
    blob = cv2.dnn.blobFromImage(img,0.00392,(416,416),(0,0,0),True, crop=False)
    
    net.setInput(blob)
    outs=net.forward(output_layers)
    
    #show info on the screen
    class_ids=[]
    confidences=[]
    boxes=[]
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id=np.argmax(scores)
            confidence=scores[class_id]
            if confidence>0.05:
                #object detected
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)
                #cv2.circle(img,(center_x,center_y),10,(0,255,0),2)
                #Rectangle coordinate
                x=int(center_x-w/2)
                y=int(center_y-h/2)

                boxes.append([x,y,w,h])
                confidences.append(float(confidence))
                class_ids.append(class_id)


    indexes=cv2.dnn.NMSBoxes(boxes,confidences,0.5,0.4)
    #print(indexes)
    font=cv2.FONT_HERSHEY_SIMPLEX
    for i in range(len(boxes)):
        if i in indexes:
            x,y,w,h=boxes[i]
            label=str(classes[class_ids[i]])
            color=colors[i]
            cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
            cv2.putText(img,label,(x,y+30),font,1,color,2)
            #print(i,label)
            labels.append(label)
            photos.append(p)
    
    cv2.imshow("Image_"+p,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    output = pd.DataFrame(
    {'photos': photos,
     'labels': labels
    })

In [14]:
output.tail(21)

,photos,labels
13,img/people1.png,person
14,img/people1.png,person
15,img/people1.png,person
16,img/people1.png,cell phone
17,img/people1.png,cell phone
18,img/people1.png,cell phone
19,img/people2.png,handbag
20,img/people2.png,backpack
21,img/people2.png,person
22,img/people2.png,person


In [15]:
output[output.labels=='person'].groupby(["photos"]).count().reset_index()

,photos,labels
0,img/people1.png,4
1,img/people2.png,11
